In [1]:
import numpy as np
import pandas as pd

In [2]:
fname = '가 나 다 라 마 바 사 아 자 차 카 타 파 하 A B C D E F G H I J K L M N O P Q R S T U V W X Y Z'.split()

In [3]:
df = pd.DataFrame()
for fn in fname:
    try:
        tmp = pd.read_csv(f'../static/data/movie{fn}.csv')
    except:
        continue
    df = pd.concat([df, tmp])
df.shape

(2960, 10)

In [4]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\n갈색 눈동자를 가진 평범한 학생 '..."


In [4]:
df.set_index('code', inplace=True)
df.reset_index(inplace=True)
df.fillna('', inplace=True)
print(df.shape)
df.head(1)


(2960, 10)


,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생..."


In [6]:
df.isna().sum()

code               0
title              0
genre              0
summering          0
first_day          0
production_year    0
movie_director     0
star_actor         0
img                0
synopsis           0
dtype: int64

In [6]:
m_type, m_kind, m_genre, m_time, m_rated, m_nation = [], [], [], [], [], []
for info in df.summering.values:
    infos = info.split('|')
    infos_len = len(infos)
    m_type.append(infos[0].strip())                     # 장편
    m_kind.append(infos[1].strip())                     # 일반영화, 예술영화..
    m_genre.append(infos[2].strip().replace(',', ''))   # 애니메이션, 드라마, 멜로
    m_nation.append(infos[-1].strip())                  # 한국
    
    # 요약 정보 개수가 다를 수 있다
    if infos_len == 6:
        m_time.append(infos[-3].strip())                # 70분
        m_rated.append(infos[-2].strip())
    elif infos_len == 5 :
        m_time.append(infos[-2].strip())                # 12세 이상 관람가
        m_rated.append('')
    elif infos_len == 4:
        m_rated.append('')
        m_time.append('')
    else:
        print(info)

len(m_type)

2960

In [7]:
df['m_type'] = m_type
df['m_kind'] = m_kind
df['m_genre'] = m_genre
df['m_nation'] = m_nation
df['m_time'] = m_time
df['m_rated'] = m_rated

In [9]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\n갈색 눈동자를 가진 평범한 학생 '...",장편,"예술,독립 영화",애니메이션 드라마 멜로/로맨스,한국,60분 57초,12세이상관람가


In [ ]:
df.tail(5)

In [8]:
indexs = df[df.summering.str.contains('멜로/로맨스') & df.summering.str.contains('청소년관람불가')].index
indexs

Int64Index([], dtype='int64')

In [9]:

df[df.star_actor.str.find('[') >= 0]

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated


In [13]:
# 감독 자리에 다른 값들이 들어갔다. 주연 배우 자리에는 데이터가 없고
# 잘못 처리했다. 일단 감독이 이상한 것은 지우고 테스트
indexs = df[df.movie_director.str.find('[') >= 0].index
indexs

Index([], dtype='int64')

In [142]:
df.drop(df.index[indexs], inplace=True)
df.set_index('code', inplace=True)
df.reset_index(inplace=True)

In [143]:
df.shape

(2746, 16)

##### 시놉시스 형태소 분석해 놓기

In [10]:
from konlpy.tag import Okt
okt = Okt()
stop_words = '''의 가 에 과 을 의 를 는 이 은 와 
            '''.split()

In [11]:
import re, string

morphs = []

for text in df.synopsis:
    text = re.sub('['+string.punctuation+'\r\n“”‘’…]', ' ', text)
    tokens = okt.morphs(text)
    morphs.append(' '.join([word for word in tokens if word not in stop_words]))

df['morphs'] = morphs


In [12]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated,morphs
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생...",장편,"예술,독립 영화",애니메이션 드라마 멜로/로맨스,한국,60분 57초,12세이상관람가,서로 몸 차라리 꽃잎 물결 가까웠다 갈색 눈동자 가진 평범한 학생 이경 여름 햇살 ...


In [13]:
df.to_csv('../static/data/movie_test.csv', index=False)

In [ ]:
for title in df.title:
    print(title)